<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#API-urls" data-toc-modified-id="API-urls-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>API-urls</a></span></li><li><span><a href="#Company-profiles" data-toc-modified-id="Company-profiles-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Company profiles</a></span></li><li><span><a href="#Quotes" data-toc-modified-id="Quotes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Quotes</a></span></li></ul></div>

**General Description**

- Code for Data Scrapping to local project space is presented in this notebook
- Data Source is specialized web-site https://financialmodelingprep.com/, which provides greatest and simple API for loading financial data and which not requires Registration and Autentification
- The data loaded in json and csv formats
- Due to the fact, that data loading takes several hours, some parts of the code for that were shown in illustrative purposes and commented for safety reason 
- loaded data have about 0.3 Gb and contains ~10k files
- due to GitHub limitations, I've put loaded files on my Google Drive with this [link](https://drive.google.com/open?id=11Zw-DvNbpc_lc3kj4AFzwESaKZ6Rs-0M)
  - raw loaded data is located in `data.rar`
  - if the project forked, this file need to be unzipped to project folder


**Data Loading Process**

- I've loaded here company profiles and quotes (prices)
- Financial Statements (on quarter basis) will be loaded only on selected (after clusterization) companies in Step-7 of the Project
- Manual Functions and Classes used here and in other Steps of the Project are imported from `project_lib.py`

- Company Profiles: some part of Data Cleaning and preprocessing for Company profiles have been done here:
  - dropping dublicates
  - company filtering
  - saving in DataFrame (csv)
  
- Company quotes: loaded only for filtered company list (from Company Profiles) as separate CSV for each company
- For safety purposes some code parts here - It is recommended to not run this code, all data already loaded and saved to GoogleDrive


**Companies filtering**  

The Project main target - is to create robust predictors for some public companies. That's why I decided to exclude from loaded tickers some composite-type indeces which could be formulated as some linear combinations of initial public companies. The following groups were excluded in filtering stage
- General Composite Indeces such as Dow Jones and S&P (full list - in `index_names` 2.2)
- Any other tickers with keyword `index`
- ETF and Mutual Funds
- Cryptocurrencies

After the filtering total tickers numbers reduced from 12 138 to 10 450


**Functions used in this notebook**
- `get_jsonparsed_data` - makes get-request for json-type data
- `find_shablons` - used for finding keyword in list of ticker names (for filtering purposes)
- `dt_to_string` - transoform datetime format into given STR-format
  - this function is used in another function: `url_for_stock`
- `url_for_stock` - used for generate API-URL for company daily quotes with given start and end dates

**Local Space Structure for loaded Data**
- base folder for loaded data: `data`
- company profiles (`profiles.csv`) - in base folder
- company daily quotes ({`company_symbol.csv`}) - in `data/quotes`
- company quartely financial statements ({`company_symbol_fsq.csv`}) - in `data/fsq`
  - this files will be loaded later, in Step-7

### API-urls

In [1]:
# 1.1. necessary modules
import numpy as np
import pandas as pd
import os
import json
import csv
import requests
import time
import warnings
from datetime import date as dt
import pickle

warnings.filterwarnings('ignore')

In [2]:
# 1.2. main API URLS
COMP_LIST = "https://financialmodelingprep.com/api/v3/company/stock/list"
PROFILE_per_COMP = f"https://financialmodelingprep.com/api/v3/company/profile/"
ETF_LIST = "https://financialmodelingprep.com/api/v3/symbol/available-etfs"
MF_LIST = "https://financialmodelingprep.com/api/v3/symbol/available-mutual-funds"
CRYPTO_LIST = "https://financialmodelingprep.com/api/v3/cryptocurrencies"
QUOTES_BASE_URL = "https://financialmodelingprep.com/api/v3/historical-price-full/"
COMMODITIES_LIST_URL = "https://financialmodelingprep.com/api/v3/symbol/available-commodities"
COMMODITIES_BASE_URL = "https://financialmodelingprep.com/api/v3/historical-price-full/commodity/"
URL_IS_Q_JSON = "https://financialmodelingprep.com/api/v3/financials/income-statement/"
URL_BS_Q_JSON = "https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/"
URL_CF_Q_JSON = "https://financialmodelingprep.com/api/v3/financials/cash-flow-statement/"

In [3]:
# 1.3. necessary functions from project_lib module
from project_lib import get_jsonparsed_data, find_shablons, dt_to_string, url_for_stock

### Company profiles

In [4]:
# 2.1. Parsing tickers list and profiles with cleaning

tickers = get_jsonparsed_data(COMP_LIST)['symbolsList']
df_all_tickers = pd.DataFrame(tickers)
df_all_tickers = df_all_tickers.drop(columns=['price'], axis=1)
name_list = df_all_tickers.name.tolist()

#print(f'total {len(name_list)} tickers parsed, including {len(name_list) - len(set(name_list))} dublicates')

In [5]:
# 2.2. Data Cleaning (taking off composite indeces and ETF)
index_names = ['Dow Jones', 'S&P', 'AMEX', 'NASDAQ', 'NYSE', 'Russel','Wilshire', 'CAC', 'DAX', 'FTSE', 
               'Nikkei', 'TSE', 'index', 'etf']

overall_set = set()
for i in index_names:
    temp_list = set(find_shablons(i, name_list))
    overall_set.update(temp_list)
overall_set = list(overall_set)

tickers_to_drop = []
for name, ticker in zip(name_list, df_all_tickers.symbol):
    if name in overall_set:
        tickers_to_drop.append(ticker)

comp_tickers_list = list(set(df_all_tickers.symbol) - set(tickers_to_drop))
#print(f'{len(comp_tickers_list)} company tickers in total')

In [6]:
# 2.3. Additional Data Cleaning (taking off ETF, Mutual Funds and CryptoCurrencies)
tickers_to_drop = []
    
for url in [ETF_LIST, MF_LIST, CRYPTO_LIST]: 
    data = get_jsonparsed_data(url)
    if url == CRYPTO_LIST:
        df = pd.DataFrame(data['cryptocurrenciesList'])
        temp_list = list(df['ticker'].values)
    else:
        df = pd.DataFrame(data)
        temp_list = list(df['symbol'].values)
    tickers_to_drop.extend(temp_list)

comp_tickers_list = list(set(comp_tickers_list) - set(tickers_to_drop))
#print(f'{len(comp_tickers_list)} company tickers in total excluding funds and crypto')

In [7]:
# 2.4 Loading company profiles
# -- Atention: it can take several hours

temp_list = []
progress_scanner_int = 500
idx = 0
total_items = len(comp_tickers_list)

start_time = time.time()
local_start_time = time.time()

for ticker in comp_tickers_list:
    idx += 1
    url = PROFILE_per_COMP+ticker
    data = get_jsonparsed_data(url)['profile']
    data['symbol'] = ticker
    temp_list.append(data)
    if idx%progress_scanner_int == 0:
        print(f'{idx} items ({round(100*idx/total_items, 1)}%) loaded ...')
        print(f'time: {time.time()-local_start_time} seconds')
        print(f'{(time.time()-local_start_time)/progress_scanner_int} seconds per 1 company')
        print(50*'.')
        local_start_time = time.time()
end_time = time.time()
total_time = end_time - start_time

df_reduced_profiles = pd.DataFrame(temp_list)
df_columns = ['symbol', 'companyName', 'sector', 'industry', 'exchange', 'description', 'ceo', 'mktCap']
df_reduced_profiles = df_reduced_profiles[df_columns]

#print(f'profiles for {len(df_reduced_profiles)} companies loaded in {total_time} sec')
#display(df_reduced_profiles.head())

In [8]:
# 2.5 Save dataset to CSV
with open('data/profiles.csv', 'w', encoding='utf-8') as f:
    df_reduced_profiles.to_csv(f, columns=df_reduced_profiles.columns) 

### Quotes

In [9]:
# 3.1. Quotes loading
# ATTENTION: it will take up to 3 hours

progress_scanner_int = 500
idx = 0
total_items = len(comp_tickers_list)

start_time = time.time()
local_start_time = time.time()
date_for_quotes = dt(2016, 1, 1)


for ticker in comp_tickers_list:
    idx += 1
    url = url_for_stock(QUOTES_BASE_URL, ticker, start_date_in_dt=date_for_quotes)
    data = get_jsonparsed_data(url)
    try:
        df = pd.DataFrame(data['historical'])[['date', 'close', 'volume']]
        filename = ticker+'_quotes.csv'
        path = f'data/quotes/{filename}'
        try:
            with open(path, 'w', encoding='utf-8') as f:
                df.to_csv(f, encoding='utf-8', sep=',', columns=df.columns) 
        except Exception:
            pass
    except Exception:
        pass
    
    if idx%progress_scanner_int == 0:
        print(f'{idx} tables ({round(100*idx/total_items, 1)}%) loaded ...')
        print(f'time: {time.time()-local_start_time} seconds')
        print(f'{(time.time()-local_start_time)/progress_scanner_int} seconds per 1 company')
        print(50*'.')
        local_start_time = time.time()

end_time = time.time()
total_time = end_time - start_time

#print(f'historical quotes for {idx} companies loaded in {total_time} sec')